In [1]:
import os

# Directorio de entrenamiento
training_path = './reuters/training'  # Asegúrate de que esta ruta es correcta

# Leer documentos
documents = []
for filename in os.listdir(training_path):
    filepath = os.path.join(training_path, filename)
    with open(filepath, 'r') as f:
        content = f.read()
        documents.append({'filename': filename, 'content': content})

print(f"Se han leído {len(documents)} documentos.")


Se han leído 7769 documentos.


In [2]:
import re

# Función para limpiar texto
def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)  # Eliminar espacios adicionales
    texto = re.sub(r'\W', ' ', texto)   # Eliminar caracteres no alfanuméricos
    texto = texto.lower()               # Convertir a minúsculas
    return texto

# Limpiar documentos
for doc in documents:
    doc['clean_content'] = limpiar_texto(doc['content'])

print("Limpieza de datos completada.")


Limpieza de datos completada.


In [3]:
import nltk
from nltk.tokenize import word_tokenize

# Descargar el paquete 'punkt' de NLTK si no está disponible
nltk.download('punkt')

# Tokenización
for doc in documents:
    doc['tokens'] = word_tokenize(doc['clean_content'])

print("Tokenización completada.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jair2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenización completada.


In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Leer stopwords
stopwords_path = './reuters/stopwords'  # Asegúrate de que esta ruta es correcta
with open(stopwords_path, 'r') as f:
    custom_stop_words = set(f.read().splitlines())

# Descargar el paquete 'stopwords' de NLTK si no está disponible
nltk.download('stopwords')
nltk_stop_words = set(stopwords.words('english'))
stop_words = custom_stop_words.union(nltk_stop_words)

print(f"Se han leído {len(stop_words)} stopwords.")

# Inicializar el stemmer
ps = PorterStemmer()

# Eliminar stopwords y aplicar stemming
for doc in documents:
    doc['tokens'] = [ps.stem(word) for word in doc['tokens'] if word not in stop_words]

print("Eliminación de stopwords y stemming completados.")


Se han leído 599 stopwords.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jair2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Eliminación de stopwords y stemming completados.


In [5]:

### Verificar contenido de los documentos


# Verificar manualmente algunos archivos
sample_files = documents[:5]  # Tomar los primeros 5 documentos para verificar
for doc in sample_files:
    print(f"Documento: {doc['filename']}")
    print(f"Contenido original: {doc['content'][:100]}...")  # Mostrar primeros 100 caracteres
    print(f"Tokens: {doc['tokens'][:10]}")  # Mostrar primeros 10 tokens
    print()


Documento: 1
Contenido original: BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating th...
Tokens: ['bahia', 'cocoa', 'review', 'shower', 'continu', 'week', 'bahia', 'cocoa', 'zone', 'allevi']

Documento: 10
Contenido original: COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES SALE
  Computer Terminal Systems Inc said
  it has com...
Tokens: ['comput', 'termin', 'system', 'lt', 'cpml', 'complet', 'sale', 'comput', 'termin', 'system']

Documento: 100
Contenido original: N.Z. TRADING BANK DEPOSIT GROWTH RISES SLIGHTLY
  New Zealand's trading bank seasonally
  adjusted d...
Tokens: ['trade', 'bank', 'deposit', 'growth', 'rise', 'slightli', 'zealand', 'trade', 'bank', 'season']

Documento: 1000
Contenido original: NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> BID
  Viacom International Inc said &lt;National
  Amu...
Tokens: ['nation', 'amus', 'up', 'viacom', 'lt', 'bid', 'viacom', 'intern', 'lt', 'nation']

Documento: 10000
Contenido original: ROGE

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Convertir los tokens de los documentos a una representación textual
clean_contents = [' '.join(doc['tokens']) for doc in documents]

# Inicializar y ajustar el vectorizador Bag of Words
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(clean_contents)

# Inicializar y ajustar el vectorizador TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(clean_contents)

print(f"Matriz BoW de dimensiones: {X_bow.shape}")
print(f"Matriz TF-IDF de dimensiones: {X_tfidf.shape}")


Matriz BoW de dimensiones: (7769, 19432)
Matriz TF-IDF de dimensiones: (7769, 19432)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
cats_path = './reuters/cats.txt' 
# Suponiendo que las categorías están disponibles y son necesarias para las etiquetas
# Si necesitas crear etiquetas desde las categorías, adapta este código
labels = [doc['cats_path'].split('.')[0] for doc in documents]  # Esto es solo un ejemplo, ajusta según tu necesidad

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, labels, test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf = train_test_split(X_tfidf, test_size=0.2, random_state=42)[0:2]  # Mismas divisiones para ambos

# Entrenar y evaluar usando BoW
clf_bow = MultinomialNB()
clf_bow.fit(X_train_bow, y_train)
y_pred_bow = clf_bow.predict(X_test_bow)
accuracy_bow = accuracy_score(y_test, y_pred_bow)

# Entrenar y evaluar usando TF-IDF
clf_tfidf = MultinomialNB()
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)

print(f"Precisión usando BoW: {accuracy_bow:.4f}")
print(f"Precisión usando TF-IDF: {accuracy_tfidf:.4f}")


Precisión usando BoW: 0.0000
Precisión usando TF-IDF: 0.0000
